In [ ]:
import open3d as o3d
import open3d.core as o3c

if o3d.__DEVICE_API__ == 'cuda':
    import open3d.cuda.pybind.t.pipelines.registration as treg
else:
    import open3d.cpu.pybind.t.pipelines.registration as treg

import numpy as np
import sys
import os
import time

# monkey patches visualization and provides helpers to load geometries
sys.path.append('..')
import open3d_tutorial as o3dtut
# change to True if you want to interact with the visualization windows
o3dtut.interactive = not "CI" in os.environ

# ICP registration
This tutorial demonstrates the ICP (Iterative Closest Point) registration algorithm. It has been a mainstay of geometric registration in both research and industry for many years. The inputs are two point clouds and an initial transformation that roughly aligns the source point cloud to the target point cloud. The output is a refined transformation that tightly aligns the two point clouds. A helper function `draw_registration_result` visualizes the alignment during the registration process. In this tutorial, we show different ICP variants, and the API for using them.

## Helper visualization function

The function below visualizes a target point cloud and a source point cloud transformed with an alignment transformation. The target point cloud and the source point cloud are painted with cyan and yellow colors respectively. The more and tighter the two point-clouds overlap with each other, the better the alignment result.

In [ ]:
def draw_registration_result(source, target, transformation):
    source_temp = source.clone()
    target_temp = target.clone()

    source_temp.transform(transformation)

    # This is patched version for tutorial rendering.
    # Use `draw` function for you application.
    o3d.visualization.draw_geometries(
        [source_temp.to_legacy(),
         target_temp.to_legacy()],
        zoom=0.4459,
        front=[0.9288, -0.2951, -0.2242],
        lookat=[1.6784, 2.0612, 1.4451],
        up=[-0.3402, -0.9189, -0.1996])

## Understanding ICP Algorithm

In general, the ICP algorithm iterates over two steps:

1. Find correspondence set $\mathcal{K}=\{(\mathbf{p}, \mathbf{q})\}$ from target point cloud $\mathbf{P}$, and source point cloud $\mathbf{Q}$ transformed with current transformation matrix $\mathbf{T}$.
2. Update the transformation $\mathbf{T}$ by minimizing an objective function $E(\mathbf{T})$ defined over the correspondence set $\mathcal{K}$.

Different variants of ICP use different objective functions $E(\mathbf{T})$ [\[BeslAndMcKay1992\]](../reference.html#beslandmckay1992) [\[ChenAndMedioni1992\]](../reference.html#chenandmedioni1992) [\[Park2017\]](../reference.html#park2017).

## Understanding ICP API

<div class="alert alert-info">
    
**Note:** 

The `Tensor based ICP implementation` API is slightly different than the [Eigen based ICP implementation](../pipelines/icp_registration.rst), to support more functionalities.
    
</div>

### Input


`PointClouds` between which the `Transformation` is to be estimated. [open3d.t.PointCloud]

- Source Tensor PointCloud. [Float32 or Float64 dtypes are supported].
- Target Tensor PointCloud. [Float32 or Float64 dtypes are supported].

<div class="alert alert-info">
    
**Note:** 

The initial alignment is usually obtained by a global registration algorithm.

</div>

In [ ]:
demo_icp_pcds = o3d.data.DemoICPPointClouds()
source = o3d.t.io.read_point_cloud(demo_icp_pcds.paths[0])
target = o3d.t.io.read_point_cloud(demo_icp_pcds.paths[1])

# For Colored-ICP `colors` attribute must be of the same dtype as `positions` and `normals` attribute.
source.point["colors"] = source.point["colors"].to(
    o3d.core.Dtype.Float32) / 255.0
target.point["colors"] = target.point["colors"].to(
    o3d.core.Dtype.Float32) / 255.0

# Initial guess transform between the two point-cloud.
# ICP algorithm requires a good initial alignment to converge efficiently.
trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
                         [-0.139, 0.967, -0.215, 0.7],
                         [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]])

draw_registration_result(source, target, trans_init)

---

### Parameters 

#### Max correspondence Distances
- This is the radius of distance from each point in the source point-cloud in which the neighbour search will try to find a corresponding point in the target point-cloud.
- It is a `double` for `icp`, and `utility.DoubleVector` for `multi-scale-icp`.
- One may typically keep this parameter between `1.0x - 3.0x` `voxel-size` for each scale.
- This parameter is most important for performance tuning, as a higher radius will take larger time (as the neighbour search will be performed over a larger radius).

In [ ]:
# For Vanilla ICP (double)

# Search distance for Nearest Neighbour Search [Hybrid-Search is used].
max_correspondence_distance = 0.07

In [ ]:
# For Multi-Scale ICP (o3d.utility.DoubleVector):

# `max_correspondence_distances` is proportional to the resolution or the `voxel_sizes`.
# In general it is recommended to use values between 1x - 3x of the corresponding `voxel_sizes`.
# We may have a higher value of the `max_correspondence_distances` for the first coarse
# scale, as it is not much expensive, and gives us more tolerance to initial alignment.
max_correspondence_distances = o3d.utility.DoubleVector([0.3, 0.14, 0.07])

#### Initial Transform from Source to Target [open3d.core.Tensor]
- Initial estimate for transformation from source to target.
- Transformation matrix Tensor of shape [4, 4] of type `Float64` on `CPU:0` device
- The initial alignment is usually obtained by a global registration algorithm. See [Global registration](../pipelines/global_registration.rst) for examples.

In [ ]:
# Initial alignment or source to target transform.
init_source_to_target = np.asarray([[0.862, 0.011, -0.507, 0.5],
                                    [-0.139, 0.967, -0.215, 0.7],
                                    [0.487, 0.255, 0.835, -1.4],
                                    [0.0, 0.0, 0.0, 1.0]])

#### Estimation Method 
- This sets the ICP method to compute the transformation between two point-clouds given the correspondences.

Options:

- **o3d.t.pipelines.registration.TransformationEstimationPointToPoint()**
    - Point to Point ICP.
- **o3d.t.pipelines.registration.TransformationEstimationPointToPlane(robust_kernel)**
    - Point to Plane ICP.
    - Requires `target point-cloud` to have `normals` attribute (of same dtype as `position` attribute).
- **o3d.t.pipelines.registration.TransformationEstimationForColoredICP(robust_kernel, lambda)**
    - Colored ICP.
    - Requires `target` point-cloud to have `normals` attribute (of same dtype as `position` attribute).
    - Requires `source` and `target` point-clouds to have `colors` attribute (of same dtype as `position` attribute).
- **o3d.t.pipelines.registration.TransformationEstimationForDopplerICP(period, lambda_doppler, ...)**
    - Doppler ICP.
    - Requires `target` point-cloud to have `normals` attribute (of same dtype as `position` attribute).
    - Requires `source` point-cloud to have `dopplers` and `directions` attributes (of same dtype as `position` attribute).
    - Requires calibration parameters: `period` (time between scans) and `transform_vehicle_to_sensor` (4x4 transformation matrix).
- **o3d.t.pipelines.registration.TransformationEstimationForGeneralizedICP(robust_kernel, epsilon)** [To be added].
    - Generalized ICP.


In [ ]:
# Select the `Estimation Method`, and `Robust Kernel` (for outlier-rejection).
estimation = treg.TransformationEstimationPointToPlane()

Estimation Method also supports `Robust Kernels`: Robust kernels are used for outlier rejection. More on this in `Robust Kernel` section.

`robust_kernel = o3d.t.pipelines.registration.robust_kernel.RobustKernel(method, scale, shape)`

Method options:

- robust_kernel.RobustKernelMethod.L2Loss
- robust_kernel.RobustKernelMethod.L1Loss
- robust_kernel.RobustKernelMethod.HuberLoss
- robust_kernel.RobustKernelMethod.CauchyLoss
- robust_kernel.RobustKernelMethod.GMLoss
- robust_kernel.RobustKernelMethod.TukeyLoss
- robust_kernel.RobustKernelMethod.GeneralizedLoss


#### ICP Convergence Criteria [relative rmse, relative fitness, max iterations]
- This sets the condition for termination or when the scale iterations can be considered to be converged. 
- If the relative (of change in value from the last iteration) rmse and fitness are equal or less than the specified value, the iterations for that scale will be considered as converged/completed.
- For `Multi-Scale ICP` it is a `list` of `ICPConvergenceCriteria`, for each scale of ICP, to provide more fine control over performance.
- One may keep the initial values of `relative_fitness` and `relative_rmse` low as we just want to get an estimate transformation, and high for later iterations to fine-tune.
- Iterations on higher-resolution is more costly (takes more time), so we want to do fewer iterations on higher resolution.

In [ ]:
# Convergence-Criteria for Vanilla ICP:

criteria = treg.ICPConvergenceCriteria(relative_fitness=0.000001,
                                       relative_rmse=0.000001,
                                       max_iteration=50)

In [ ]:
# List of Convergence-Criteria for Multi-Scale ICP:

# We can control `ConvergenceCriteria` of each `scale` individually.
# We want to keep `relative_fitness` and `relative_rmse` high (more error tolerance)
# for initial scales, i.e. we will be happy to consider ICP converged, when difference
# between 2 successive iterations for that scale is smaller than this value.
# We expect less accuracy (more error tolerance) initial coarse-scale iteration,
# and want our later scale convergence to be more accurate (less error tolerance).
criteria_list = [
    treg.ICPConvergenceCriteria(relative_fitness=0.0001,
                                relative_rmse=0.0001,
                                max_iteration=20),
    treg.ICPConvergenceCriteria(0.00001, 0.00001, 15),
    treg.ICPConvergenceCriteria(0.000001, 0.000001, 10)
]

#### Voxel Sizes
- It is the voxel size (lower voxel size corresponds to higher resolution), for each scale of multi-scale ICP.
- We want to perform initial iterations on a coarse point-cloud (low-resolution or small voxel size) (as it is more time-efficient, and avoids local-minima), and then move to a dense point-cloud (high-resolution or small voxel size. Therefore the voxel sizes must be strictly decreasing order.

In [ ]:
# Vanilla ICP
voxel_size = 0.025

In [ ]:
# Lower `voxel_size` is equivalent to higher resolution,
# and we want to perform iterations from coarse to dense resolution,
# therefore `voxel_sizes` must be in strictly decressing order.
voxel_sizes = o3d.utility.DoubleVector([0.1, 0.05, 0.025])

#### Get Iteration-wise registration result using callback lambda function

Optional lambda function, saves string to tensor dictionary of attributes such as "iteration_index", "scale_index", "scale_iteration_index", "inlier_rmse", "fitness", "transformation", on CPU device, updated after each iteration.

In [ ]:
# Example callback_after_iteration lambda function:
callback_after_iteration = lambda updated_result_dict : print("Iteration Index: {}, Fitness: {}, Inlier RMSE: {},".format(
    updated_result_dict["iteration_index"].item(),
    updated_result_dict["fitness"].item(),
    updated_result_dict["inlier_rmse"].item()))

---

## Vanilla ICP Example

### 1. Set Inputs and Parameters

In [ ]:
# Input point-clouds
demo_icp_pcds = o3d.data.DemoICPPointClouds()
source = o3d.t.io.read_point_cloud(demo_icp_pcds.paths[0])
target = o3d.t.io.read_point_cloud(demo_icp_pcds.paths[1])

In [ ]:
# Search distance for Nearest Neighbour Search [Hybrid-Search is used].
max_correspondence_distance = 0.07

# Initial alignment or source to target transform.
init_source_to_target = np.asarray([[0.862, 0.011, -0.507, 0.5],
                                    [-0.139, 0.967, -0.215, 0.7],
                                    [0.487, 0.255, 0.835, -1.4],
                                    [0.0, 0.0, 0.0, 1.0]])

# Select the `Estimation Method`, and `Robust Kernel` (for outlier-rejection).
estimation = treg.TransformationEstimationPointToPlane()

# Convergence-Criteria for Vanilla ICP
criteria = treg.ICPConvergenceCriteria(relative_fitness=0.000001,
                                       relative_rmse=0.000001,
                                       max_iteration=50)
# Down-sampling voxel-size.
voxel_size = 0.025

# Save iteration wise `fitness`, `inlier_rmse`, etc. to analyse and tune result.
save_loss_log = True

### 2. Get Registration Result from ICP

In [ ]:
s = time.time()

registration_icp = treg.icp(source, target, max_correspondence_distance,
                            init_source_to_target, estimation, criteria,
                            voxel_size, callback_after_iteration)

icp_time = time.time() - s
print("Time taken by ICP: ", icp_time)
print("Inlier Fitness: ", registration_icp.fitness)
print("Inlier RMSE: ", registration_icp.inlier_rmse)

draw_registration_result(source, target, registration_icp.transformation)

---
Now let's try with poor initial initialisation

In [ ]:
init_source_to_target = o3d.core.Tensor.eye(4, o3d.core.Dtype.Float32)
max_correspondence_distance = 0.07

In [ ]:
s = time.time()

registration_icp = treg.icp(source, target, max_correspondence_distance,
                            init_source_to_target, estimation, criteria,
                            voxel_size)

icp_time = time.time() - s
print("Time taken by ICP: ", icp_time)
print("Inlier Fitness: ", registration_icp.fitness)
print("Inlier RMSE: ", registration_icp.inlier_rmse)

draw_registration_result(source, target, registration_icp.transformation)

As we can see, poor initial alignment might fail ICP convergence

Having large `max_correspondence_distance` might resolve this issue. But it will take longer to process.

In [ ]:
init_source_to_target = o3d.core.Tensor.eye(4, o3c.float32)
max_correspondence_distance = 0.5

In [ ]:
s = time.time()
# It is highly recommended to down-sample the point-cloud before using
# ICP algorithm, for better performance.

registration_icp = treg.icp(source, target, max_correspondence_distance,
                            init_source_to_target, estimation, criteria,
                            voxel_size)

icp_time = time.time() - s
print("Time taken by ICP: ", icp_time)
print("Inlier Fitness: ", registration_icp.fitness)
print("Inlier RMSE: ", registration_icp.inlier_rmse)

draw_registration_result(source, target, registration_icp.transformation)

We may resolve the above issues and get even better accuracy by using `Multi-Scale ICP`

---

## Multi-Scale ICP Example

Problems with using Vanilla-ICP (previous version):

- Running the ICP algorithm on dense point-clouds is very slow. 
- It requires good initial alignment:
    - If the point-cloud is not well aligned, the convergence might get stuck in local-minima in initial iterations.
    - We need to have a larger `max_correspondence_distance` if the aligned point cloud does not have sufficient overlaps.  
- If point-cloud is heavily down-sampled (coarse), the obtained result will not be accurate. 

These drawbacks can be solved using Multi-Scale ICP.
In Multi-Scale ICP, we perform the initial iterations on coarse point-cloud to get a better estimate of initial alignment and use this alignment for convergence on a more dense point cloud. ICP on coarse point cloud is in-expensive, and allows us to use a larger `max_correspondence_distance`. It is also less likely for the convergence to get stuck in local minima. As we get a good estimate, it takes fewer iterations on dense point-cloud to converge to a more accurate transform. 

It is recommended to use `Multi-Scale ICP` over `ICP`, for efficient convergence, especially for large point clouds. 

### 1. Set Inputs and Parameters

In [ ]:
# Input point-clouds
demo_icp_pcds = o3d.data.DemoICPPointClouds()
source = o3d.t.io.read_point_cloud(demo_icp_pcds.paths[0])
target = o3d.t.io.read_point_cloud(demo_icp_pcds.paths[1])

In [ ]:
voxel_sizes = o3d.utility.DoubleVector([0.1, 0.05, 0.025])

# List of Convergence-Criteria for Multi-Scale ICP:
criteria_list = [
    treg.ICPConvergenceCriteria(relative_fitness=0.0001,
                                relative_rmse=0.0001,
                                max_iteration=20),
    treg.ICPConvergenceCriteria(0.00001, 0.00001, 15),
    treg.ICPConvergenceCriteria(0.000001, 0.000001, 10)
]

# `max_correspondence_distances` for Multi-Scale ICP (o3d.utility.DoubleVector):
max_correspondence_distances = o3d.utility.DoubleVector([0.3, 0.14, 0.07])

# Initial alignment or source to target transform.
init_source_to_target = o3d.core.Tensor.eye(4, o3d.core.Dtype.Float32)

# Select the `Estimation Method`, and `Robust Kernel` (for outlier-rejection).
estimation = treg.TransformationEstimationPointToPlane()

# Save iteration wise `fitness`, `inlier_rmse`, etc. to analyse and tune result.
callback_after_iteration = lambda loss_log_map : print("Iteration Index: {}, Scale Index: {}, Scale Iteration Index: {}, Fitness: {}, Inlier RMSE: {},".format(
    loss_log_map["iteration_index"].item(),
    loss_log_map["scale_index"].item(),
    loss_log_map["scale_iteration_index"].item(),
    loss_log_map["fitness"].item(),
    loss_log_map["inlier_rmse"].item()))

### 2. Get Registration Result from Multi-Scale ICP

In [ ]:
# Setting Verbosity to Debug, helps in fine-tuning the performance.
# o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)

s = time.time()

registration_ms_icp = treg.multi_scale_icp(source, target, voxel_sizes,
                                           criteria_list,
                                           max_correspondence_distances,
                                           init_source_to_target, estimation,
                                           callback_after_iteration)

ms_icp_time = time.time() - s
print("Time taken by Multi-Scale ICP: ", ms_icp_time)
print("Inlier Fitness: ", registration_ms_icp.fitness)
print("Inlier RMSE: ", registration_ms_icp.inlier_rmse)

draw_registration_result(source, target, registration_ms_icp.transformation)

---

### Multi-Scale ICP on CUDA device Example

In [ ]:
# The algorithm runs on the same device as the source and target point-cloud.
source_cuda = source.cuda(0)
target_cuda = target.cuda(0)

In [ ]:
s = time.time()

registration_ms_icp = treg.multi_scale_icp(source_cuda, target_cuda,
                                           voxel_sizes, criteria_list,
                                           max_correspondence_distances,
                                           init_source_to_target, estimation)

ms_icp_time = time.time() - s
print("Time taken by Multi-Scale ICP: ", ms_icp_time)
print("Inlier Fitness: ", registration_ms_icp.fitness)
print("Inlier RMSE: ", registration_ms_icp.inlier_rmse)

draw_registration_result(source.cpu(), target.cpu(),
                         registration_ms_icp.transformation)

---

## Case of `no correspondences`.
In case of `no_correspondences` the `fitness` and `inlier_rmse` is `0`. 

In [ ]:
max_correspondence_distance = 0.02

init_source_to_target = np.asarray([[1.0, 0.0, 0.0, 5], [0.0, 1.0, 0.0, 7],
                                    [0.0, 0.0, 1.0, 10], [0.0, 0.0, 0.0, 1.0]])

registration_icp = treg.icp(source, target, max_correspondence_distance,
                            init_source_to_target)

print("Inlier Fitness: ", registration_icp.fitness)
print("Inlier RMSE: ", registration_icp.inlier_rmse)
print("Transformation: \n", registration_icp.transformation)

if registration_icp.fitness == 0 and registration_icp.inlier_rmse == 0:
    print("ICP Convergence Failed, as no correspondence were found")

---

## Information Matrix

`Information Matrix` gives us further information about how well the point-clouds are aligned. 

In [ ]:
information_matrix = treg.get_information_matrix(
    source, target, max_correspondence_distances[2],
    registration_ms_icp.transformation)

print(information_matrix)

---

Now that we have a basic understanding of the ICP algorithm and the API, let's experiment with the different versions to understand the difference

## Initial Alignment

In [ ]:
demo_icp_pcds = o3d.data.DemoICPPointClouds()
source = o3d.t.io.read_point_cloud(demo_icp_pcds.paths[0])
target = o3d.t.io.read_point_cloud(demo_icp_pcds.paths[1])

# Initial guess transform between the two point-cloud.
# ICP algorithm requires a good initial alignment to converge efficiently.
trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
                         [-0.139, 0.967, -0.215, 0.7],
                         [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]])

draw_registration_result(source, target, trans_init)

In [ ]:
# Search distance for Nearest Neighbour Search [Hybrid-Search is used].
max_correspondence_distance = 0.02

print("Initial alignment")
evaluation = treg.evaluate_registration(source, target,
                                        max_correspondence_distance, trans_init)

print("Fitness: ", evaluation.fitness)
print("Inlier RMSE: ", evaluation.inlier_rmse)

---

## Point-To-Point ICP Registration 

We first show a point-to-point ICP algorithm [\[BeslAndMcKay1992\]](../reference.html#beslandmckay1992) using the objective

\begin{equation}
E(\mathbf{T}) = \sum_{(\mathbf{p},\mathbf{q})\in\mathcal{K}}\|\mathbf{p} - \mathbf{T}\mathbf{q}\|^{2}
\end{equation}

The class `TransformationEstimationPointToPoint` provides functions to compute the residuals and Jacobian matrices of the point-to-point ICP objective.

### 1. Set Inputs and Parameters

In [ ]:
# Input point-clouds
demo_icp_pcds = o3d.data.DemoICPPointClouds()
source = o3d.t.io.read_point_cloud(demo_icp_pcds.paths[0])
target = o3d.t.io.read_point_cloud(demo_icp_pcds.paths[1])

In [ ]:
# Select the `Estimation Method`, and `Robust Kernel` (for outlier-rejection).
estimation = treg.TransformationEstimationPointToPoint()

In [ ]:
# Search distance for Nearest Neighbour Search [Hybrid-Search is used].
max_correspondence_distance = 0.02

# Initial alignment or source to target transform.
init_source_to_target = trans_init

# Convergence-Criteria for Vanilla ICP
criteria = treg.ICPConvergenceCriteria(relative_fitness=0.0000001,
                                       relative_rmse=0.0000001,
                                       max_iteration=30)

# Down-sampling voxel-size. If voxel_size < 0, original scale is used.
voxel_size = -1

# Save iteration wise `fitness`, `inlier_rmse`, etc. to analyse and tune result.
save_loss_log = True

In [ ]:
print("Apply Point-to-Point ICP")
s = time.time()

reg_point_to_point = treg.icp(source, target, max_correspondence_distance,
                              init_source_to_target, estimation, criteria,
                              voxel_size)

icp_time = time.time() - s
print("Time taken by Point-To-Point ICP: ", icp_time)
print("Fitness: ", reg_point_to_point.fitness)
print("Inlier RMSE: ", reg_point_to_point.inlier_rmse)

draw_registration_result(source, target, reg_point_to_point.transformation)

The fitness score increases from `0.174722` to `0.372474`. The inlier_rmse reduces from `0.011771` to `0.007761`. 
By default, icp runs until convergence or reaches a maximum number of iterations (30 by default). 
It can be changed to allow more computation time and to improve the results further.

In [ ]:
criteria = treg.ICPConvergenceCriteria(relative_fitness=0.0000001,
                                       relative_rmse=0.0000001,
                                       max_iteration=1000)

In [ ]:
print("Apply Point-to-Point ICP")
s = time.time()

reg_point_to_point = treg.icp(source, target, max_correspondence_distance,
                              init_source_to_target, estimation, criteria,
                              voxel_size)

icp_time = time.time() - s
print("Time taken by Point-To-Point ICP: ", icp_time)
print("Fitness: ", reg_point_to_point.fitness)
print("Inlier RMSE: ", reg_point_to_point.inlier_rmse)

draw_registration_result(source, target, reg_point_to_point.transformation)

The final alignment is tight. The fitness score improves to `0.620972`. The inlier_rmse reduces to `0.006581`.

---

## Point-to-Plane ICP Registration
The point-to-plane ICP algorithm [\[ChenAndMedioni1992\]](../reference.html#chenandmedioni1992) uses a different objective function

\begin{equation}
E(\mathbf{T}) = \sum_{(\mathbf{p},\mathbf{q})\in\mathcal{K}}\big((\mathbf{p} - \mathbf{T}\mathbf{q})\cdot\mathbf{n}_{\mathbf{p}}\big)^{2},
\end{equation}

where $\mathbf{n}_{\mathbf{p}}$ is the normal of point $\mathbf{p}$. [\[Rusinkiewicz2001\]](../reference.html#rusinkiewicz2001) has shown that the point-to-plane ICP algorithm has a faster convergence speed than the point-to-point ICP algorithm.

The class `TransformationEstimationPointToPlane` provides functions to compute the residuals and Jacobian matrices of the point-to-plane ICP objective. 

In [ ]:
estimation = treg.TransformationEstimationPointToPlane()

criteria = treg.ICPConvergenceCriteria(relative_fitness=0.0000001,
                                       relative_rmse=0.0000001,
                                       max_iteration=30)

In [ ]:
print("Apply Point-to-Plane ICP")
s = time.time()

reg_point_to_plane = treg.icp(source, target, max_correspondence_distance,
                              init_source_to_target, estimation, criteria,
                              voxel_size)

icp_time = time.time() - s
print("Time taken by Point-To-Plane ICP: ", icp_time)
print("Fitness: ", reg_point_to_plane.fitness)
print("Inlier RMSE: ", reg_point_to_plane.inlier_rmse)

draw_registration_result(source, target, reg_point_to_plane.transformation)

The point-to-plane ICP reaches tight alignment within 30 iterations (a `fitness` score of 0.620972 and an `inlier_rmse` score of 0.006581).

---

## Colored ICP Registration
This tutorial demonstrates an ICP variant that uses both geometry and color for registration. It implements the algorithm of [\[Park2017\]](../reference.html#park2017). The color information locks the alignment along the tangent plane. Thus this algorithm is more accurate and more robust than prior point cloud registration algorithms, while the running speed is comparable to that of ICP registration. 

In [ ]:
# Overriding visualization function, according to best camera view for colored-icp sample data.
def draw_registration_result(source, target, transformation):
    source_temp = source.clone()
    target_temp = target.clone()

    source_temp.transform(transformation)

    # This is patched version for tutorial rendering.
    # Use `draw` function for you application.
    o3d.visualization.draw_geometries(
        [source_temp.to_legacy(),
         target_temp.to_legacy()],
        zoom=0.5,
        front=[-0.2458, -0.8088, 0.5342],
        lookat=[1.7745, 2.2305, 0.9787],
        up=[0.3109, -0.5878, -0.7468])

In [ ]:
print("1. Load two point clouds and show initial pose")
demo_cicp_pcds = o3d.data.DemoColoredICPPointClouds()
source = o3d.t.io.read_point_cloud(demo_cicp_pcds.paths[0])
target = o3d.t.io.read_point_cloud(demo_cicp_pcds.paths[1])

# For Colored-ICP `colors` attribute must be of the same dtype as `positions` and `normals` attribute.
source.point["colors"] = source.point["colors"].to(
    o3d.core.Dtype.Float32) / 255.0
target.point["colors"] = target.point["colors"].to(
    o3d.core.Dtype.Float32) / 255.0

# draw initial alignment
current_transformation = np.identity(4)
draw_registration_result(source, target, current_transformation)

## Setting baseline with point-to-plane registration
We first run Point-to-plane ICP as a baseline approach. The visualization below shows misaligned green triangle textures. This is because a geometric constraint does not prevent two planar surfaces from slipping.

In [ ]:
estimation = treg.TransformationEstimationPointToPlane()

In [ ]:
max_correspondence_distance = 0.02
init_source_to_target = np.identity(4)

In [ ]:
print("Apply Point-to-Plane ICP")
s = time.time()

reg_point_to_plane = treg.icp(source, target, max_correspondence_distance,
                              init_source_to_target, estimation)

icp_time = time.time() - s
print("Time taken by Point-To-Plane ICP: ", icp_time)
print("Fitness: ", reg_point_to_plane.fitness)
print("Inlier RMSE: ", reg_point_to_plane.inlier_rmse)

draw_registration_result(source, target, reg_point_to_plane.transformation)

## Colored Registration
The core function for colored point cloud registration is `registration_colored_icp`. Following [\[Park2017\]](../reference.html#park2017), it runs ICP iterations (see [Point-to-point ICP](../pipelines/icp_registration.ipynb#Point-to-point-ICP) for details) with a joint optimization objective

\begin{equation}
E(\mathbf{T}) = (1-\delta)E_{C}(\mathbf{T}) + \delta E_{G}(\mathbf{T})
\end{equation}

where $\mathbf{T}$ is the transformation matrix to be estimated. $E_{C}$ and $E_{G}$ are the photometric and geometric terms, respectively. $\delta\in[0,1]$ is a weight parameter that has been determined empirically.

The geometric term $E_{G}$ is the same as the [Point-to-plane ICP](../pipelines/icp_registration.ipynb#Point-to-plane-ICP) objective

\begin{equation}
E_{G}(\mathbf{T}) = \sum_{(\mathbf{p},\mathbf{q})\in\mathcal{K}}\big((\mathbf{p} - \mathbf{T}\mathbf{q})\cdot\mathbf{n}_{\mathbf{p}}\big)^{2},
\end{equation}

where $\mathcal{K}$ is the correspondence set in the current iteration. $\mathbf{n}_{\mathbf{p}}$ is the normal of point $\mathbf{p}$.

The color term $E_{C}$ measures the difference between the color of point $\mathbf{q}$ (denoted as $C(\mathbf{q})$) and the color of its projection on the tangent plane of $\mathbf{p}$.

\begin{equation}
E_{C}(\mathbf{T}) = \sum_{(\mathbf{p},\mathbf{q})\in\mathcal{K}}\big(C_{\mathbf{p}}(\mathbf{f}(\mathbf{T}\mathbf{q})) - C(\mathbf{q})\big)^{2},
\end{equation}

where $C_{\mathbf{p}}(\cdot)$ is a precomputed function continuously defined on the tangent plane of $\mathbf{p}$. Function$\mathbf{f}(\cdot)$ projects a 3D point to the tangent plane. For more details, refer to [\[Park2017\]](../reference.html#park2017).

To further improve efficiency, [\[Park2017\]](../reference.html#park2017) proposes a multi-scale registration scheme. 

In [ ]:
estimation = treg.TransformationEstimationForColoredICP()

In [ ]:
current_transformation = np.identity(4)

criteria_list = [
    treg.ICPConvergenceCriteria(relative_fitness=0.0001,
                                relative_rmse=0.0001,
                                max_iteration=50),
    treg.ICPConvergenceCriteria(0.00001, 0.00001, 30),
    treg.ICPConvergenceCriteria(0.000001, 0.000001, 14)
]

max_correspondence_distances = o3d.utility.DoubleVector([0.08, 0.04, 0.02])

voxel_sizes = o3d.utility.DoubleVector([0.04, 0.02, 0.01])

In [ ]:
# colored pointcloud registration
# This is implementation of following paper
# J. Park, Q.-Y. Zhou, V. Koltun,
# Colored Point Cloud Registration Revisited, ICCV 2017

print("Colored point cloud registration")
s = time.time()

reg_multiscale_icp = treg.multi_scale_icp(source, target, voxel_sizes,
                                          criteria_list,
                                          max_correspondence_distances,
                                          init_source_to_target, estimation)

icp_time = time.time() - s
print("Time taken by Colored ICP: ", icp_time)
print("Fitness: ", reg_point_to_plane.fitness)
print("Inlier RMSE: ", reg_point_to_plane.inlier_rmse)

draw_registration_result(source, target, reg_multiscale_icp.transformation)

---

## Doppler ICP Registration
This tutorial demonstrates an ICP variant that uses both geometry and Doppler velocity measurements for registration. It implements the algorithm of [\[Hexsel2022\]](https://arxiv.org/abs/2201.11944). Doppler ICP leverages instantaneous radial velocity measurements from range sensors (such as FMCW LiDAR) to improve registration accuracy and achieve faster convergence compared to classical point-to-plane ICP. The Doppler information provides additional constraints that help disambiguate motion, making this algorithm more accurate, robust, and faster-converging than geometric-only ICP methods, especially in feature-denied environments like hallways, tunnels, highways, and bridges. 

![Comparison of tunnel reconstructions using point-to-plane ICP (left) and Doppler ICP (right) with measurements collected by an FMCW LiDAR.](images/doppler_icp.jpg)

### Understanding Doppler ICP

Doppler ICP combines geometric and Doppler velocity constraints in a joint optimization objective:

\begin{equation}
E(\mathbf{T}) = (1-\lambda)E_{G}(\mathbf{T}) + \lambda E_{D}(\mathbf{T})
\end{equation}

where $\mathbf{T}$ is the transformation matrix to be estimated, $\lambda \in [0,1]$ is a weight parameter, $E_{G}$ is the geometric term, and $E_{D}$ is the Doppler term.

The geometric term $E_{G}$ is the same as the [Point-to-plane ICP](../pipelines/icp_registration.ipynb#Point-to-plane-ICP) objective:

\begin{equation}
E_{G}(\mathbf{T}) = \sum_{(\mathbf{p},\mathbf{q})\in\mathcal{K}}\big((\mathbf{p} - \mathbf{T}\mathbf{q})\cdot\mathbf{n}_{\mathbf{p}}\big)^{2},
\end{equation}

where $\mathcal{K}$ is the correspondence set and $\mathbf{n}_{\mathbf{p}}$ is the normal of point $\mathbf{p}$.

The Doppler term $E_{D}$ measures the consistency between the measured Doppler velocity and the velocity predicted by the estimated transformation:

\begin{equation}
E_{D}(\mathbf{T}) = \sum_{(\mathbf{p},\mathbf{q})\in\mathcal{K}}\big(d_{\mathbf{q}} - \mathbf{v}(\mathbf{T})\cdot\hat{\mathbf{r}}_{\mathbf{q}}\big)^{2},
\end{equation}

where $d_{\mathbf{q}}$ is the measured Doppler velocity at point $\mathbf{q}$, $\mathbf{v}(\mathbf{T})$ is the velocity implied by transformation $\mathbf{T}$, and $\hat{\mathbf{r}}_{\mathbf{q}}$ is the unit direction vector from the sensor to point $\mathbf{q}$.

By jointly optimizing these complementary constraints, Doppler ICP achieves **faster convergence** compared to classical point-to-plane ICP that solely relies on geometric residuals. The Doppler term provides additional information that helps guide the optimization toward the correct solution more efficiently, particularly in feature-denied environments where geometry alone is ambiguous.

For more details, refer to [\[Hexsel2022\]](https://arxiv.org/abs/2201.11944).

### Requirements

Doppler ICP requires:

- **Source point cloud** with:
  - `dopplers` attribute: Measured Doppler or radial velocities for each point.
  - `directions` attribute: Unit direction vectors from sensor to each point (normalized point positions). The direction vectors must correspond the raw range measurements (without any motion compensation or undistortion applied).
  
- **Target point cloud** with:
  - `normals` attribute: Surface normals for each point.
  
- **Calibration parameters**:
  - `period`: Time period (in seconds) between source and target scans.
  - `transform_vehicle_to_sensor`: 4x4 transformation matrix from vehicle frame to sensor frame. This is optional.

In [ ]:
print("1. Load Doppler ICP dataset")
import json

demo_sequence = o3d.data.DemoDopplerICPSequence()

# Load point clouds
source_idx = 0
target_idx = 1
source = o3d.t.io.read_point_cloud(demo_sequence.paths[source_idx])
target = o3d.t.io.read_point_cloud(demo_sequence.paths[target_idx])

# Load calibration parameters
with open(demo_sequence.calibration_path) as f:
    calibration_data = json.load(f)

transform_vehicle_to_sensor = np.array(
    calibration_data['transform_vehicle_to_sensor']).reshape(4, 4)
period = calibration_data['period']

print(f"Loaded source: {demo_sequence.paths[source_idx]}")
print(f"Loaded target: {demo_sequence.paths[target_idx]}")
print(f"Time period between scans: {period:.4f} seconds")

In [ ]:
print("2. Prepare point clouds for Doppler ICP")

# Setup device
dtype = o3d.core.float32
device = o3d.core.Device('CPU:0')

# Downsample point clouds
source_in_S = source.uniform_down_sample(5)
target_in_S = target.uniform_down_sample(5)

# Transform from sensor frame (S) to vehicle frame (V)
source_in_V = source_in_S.to(device).transform(transform_vehicle_to_sensor)
target_in_V = target_in_S.to(device).transform(transform_vehicle_to_sensor)

# Compute normals for target (required for point-to-plane ICP)
target_in_V.estimate_normals(radius=10.0, max_nn=30)

# Compute direction vectors for source (required for Doppler ICP)
# Directions are unit vectors from sensor to each point (in sensor frame)
directions = source_in_S.point.positions.numpy()
norms = np.tile(np.linalg.norm(directions, axis=1), (3, 1)).T
directions = directions / norms
source_in_V.point['directions'] = o3d.core.Tensor(directions, dtype, device)

# Note: The source point cloud should already have 'dopplers' attribute
# from the dataset. If not, you would need to add it here.
print("Source point cloud attributes:", source_in_V.point.keys())
print("Target point cloud attributes:", target_in_V.point.keys())

### Setting baseline with point-to-plane registration

We first run Point-to-plane ICP as a baseline approach for comparison.

In [ ]:
# Setup robust kernel
kernel = treg.robust_kernel.RobustKernel(
    treg.robust_kernel.RobustKernelMethod.TukeyLoss,
    scaling_parameter=0.5)

# Setup convergence criteria
criteria = treg.ICPConvergenceCriteria(
    relative_fitness=1e-6,
    relative_rmse=1e-6,
    max_iteration=200)

# Point-to-plane ICP estimator
estimator_p2l = treg.TransformationEstimationPointToPlane(kernel)

# Initial transformation
init_transform = o3d.core.Tensor(np.eye(4), device=device, dtype=dtype)

# Run point-to-plane ICP
max_neighbor_distance = 0.3
print("Apply Point-to-Plane ICP")
s = time.time()

reg_p2l = treg.icp(source_in_V, target_in_V, max_neighbor_distance,
                    init_transform, estimator_p2l, criteria)

p2l_time = time.time() - s
print(f"Time taken by Point-to-Plane ICP: {p2l_time:.4f} seconds")
print(f"Fitness: {reg_p2l.fitness:.6f}")
print(f"Inlier RMSE: {reg_p2l.inlier_rmse:.6f}")
print(f"Iterations: {reg_p2l.num_iterations}")

### Doppler ICP Registration

Now we run Doppler ICP using the same parameters. The Doppler term provides additional constraints that help improve registration accuracy and achieve faster convergence compared to point-to-plane ICP.

The `TransformationEstimationForDopplerICP` class provides several parameters to control the registration behavior:

#### Core Parameters

- **`period`** (double, default: 0.1): Time period in seconds between the source and target point clouds. This is used to compute the velocity implied by the transformation. For sequential scans, multiply the base period by the frame difference (e.g., `period * (target_idx - source_idx)`).

- **`lambda_doppler`** (double, default: 0.01): Weight parameter `λ ∈ [0, 1]` that balances the geometric and Doppler terms in the objective function `(1-λ)E_G + λE_D`. A smaller value (e.g., 0.01) emphasizes geometric constraints, while a larger value increases the influence of Doppler measurements. Emprirically 0.01 is a good default value.

#### Outlier Rejection Parameters

- **`reject_dynamic_outliers`** (bool, default: false): Whether to reject dynamic point outlier correspondences. When enabled, correspondences with large Doppler errors (indicating moving objects) are pruned from the optimization.

- **`doppler_outlier_threshold`** (double, default: 2.0): Threshold for Doppler error (in m/s) above which correspondences are considered outliers and rejected. Only used when `reject_dynamic_outliers` is enabled and `outlier_rejection_min_iteration` is reached.

- **`outlier_rejection_min_iteration`** (size_t, default: 2): Number of ICP iterations after which outlier rejection is enabled. This allows the algorithm to establish initial correspondences before applying outlier filtering.

In [ ]:
# Setup Doppler ICP estimator
# Compute actual period: time between scans (multiply base period by frame difference)
actual_period = period * (target_idx - source_idx)

# Convert calibration transform to tensor
transform_vehicle_to_sensor_tensor = o3d.core.Tensor(
    transform_vehicle_to_sensor, dtype=dtype, device=device)

# Create Doppler ICP estimator with all parameters
estimator_dicp = treg.TransformationEstimationForDopplerICP(
    # Core parameters
    period=actual_period,  # Time period (seconds) between source and target scans
    lambda_doppler=0.01,   # Weight λ ∈ [0,1] for Doppler term: (1-λ)E_G + λE_D

    # Outlier rejection parameters
    reject_dynamic_outliers=False,  # Set True to reject moving object correspondences
    doppler_outlier_threshold=2.0,  # Doppler error threshold (m/s) for outlier rejection
    outlier_rejection_min_iteration=2,  # Enable outlier rejection after N iterations

    # Robust loss parameters
    geometric_robust_loss_min_iteration=0,  # Enable robust loss for geometry from start (iteration 0)
    doppler_robust_loss_min_iteration=2,    # Enable robust loss for Doppler after 2 iterations

    # Robust kernels for outlier rejection
    geometric_kernel=kernel,  # Robust kernel for geometric term (TukeyLoss with scale=0.5)
    doppler_kernel=kernel,    # Robust kernel for Doppler term (same as geometric)

    # Calibration
    transform_vehicle_to_sensor=transform_vehicle_to_sensor_tensor  # Vehicle-to-sensor transform
)

# Run Doppler ICP
print("Apply Doppler ICP")
s = time.time()

reg_dicp = treg.icp(source_in_V, target_in_V, max_neighbor_distance,
                    init_transform, estimator_dicp, criteria)

dicp_time = time.time() - s
print(f"Time taken by Doppler ICP: {dicp_time:.4f} seconds")
print(f"Fitness: {reg_dicp.fitness:.6f}")
print(f"Inlier RMSE: {reg_dicp.inlier_rmse:.6f}")
print(f"Iterations: {reg_dicp.num_iterations}")

In [ ]:
# Compare results
print("\nComparison:")
print(f"{'Method':<25} {'Fitness':<12} {'RMSE':<12} {'Iterations':<12} {'Time (s)':<12}")
print("-" * 75)
print(f"{'Point-to-Plane ICP':<25} {reg_p2l.fitness:<12.6f} {reg_p2l.inlier_rmse:<12.6f} {reg_p2l.num_iterations:<12} {p2l_time:<12.4f}")
print(f"{'Doppler ICP':<25} {reg_dicp.fitness:<12.6f} {reg_dicp.inlier_rmse:<12.6f} {reg_dicp.num_iterations:<12} {dicp_time:<12.4f}")

# Display transformations
np.set_printoptions(suppress=True, precision=4)
print("\nPoint-to-Plane ICP transformation:")
print(reg_p2l.transformation.numpy())
print("\nDoppler ICP transformation:")
print(reg_dicp.transformation.numpy())